<a href="https://colab.research.google.com/github/debaghtk/finetuning-poc/blob/main/finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%pip install transformers datasets torch accelerate

download model tokenizer and dataset

In [2]:
from transformers import AutoModelForCausalLM as AutoModel, AutoTokenizer
from datasets import load_dataset

model_name = "bigscience/bloomz-560m"

model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
dataset = load_dataset("ought/raft", "twitter_complaints")

Extracting data files:   0%|          | 0/11 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/50 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3399 [00:00<?, ? examples/s]

doing inference like TDD

In [3]:
inputs = tokenizer(
    "Tweet text: @nationalgridus I have no water and the bill is current and paid. Can you do something about it? Label:",
    padding=True,
    max_length=64,
    truncation=True,
    return_tensors="pt",
)

output = model.generate(**inputs)
print(tokenizer.batch_decode(output, skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 27, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


['Tweet text: @nationalgridus I have no water and the bill is current and paid. Can you do something about it? Label: National']


preprocessing of the data
- add text_label key, values - complaint, no complaint

In [4]:
dataset = load_dataset("ought/raft", "twitter_complaints")
classes = [k.replace("_", " ") for k in dataset["train"].features["Label"].names]
dataset = dataset.map(
    lambda x: {"label_ids": tokenizer(classes[x["Label"]], pad_to_max_length=True, max_length=64)["input_ids"]},
)
dataset = dataset.map(
    lambda x: {"model_input": f'Tweet text: {x["Tweet text"]}, Label:'},
)
dataset = dataset.remove_columns(["Label"])
# dataset = dataset.remove_columns(["Labels"])
# dataset = dataset.remove_columns(["labels"])
# dataset = dataset.remove_columns(["label"])
for i in range(len(dataset["test"])):
  if len(dataset["test"][i]["label_ids"])!=64:
    print("gadbad")

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Map:   0%|          | 0/3399 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/3399 [00:00<?, ? examples/s]

tokenize data

In [5]:
dataset = dataset.map(
    lambda x: tokenizer(x["model_input"], pad_to_max_length=True, max_length=64),
    batched=True,
)
for i in range(len(dataset["train"])):
  if len(dataset["train"][i]["input_ids"])!=len(dataset["train"][i]["label_ids"]):
    print("gadbad")

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/3399 [00:00<?, ? examples/s]

split data into train, test, validation

In [6]:
train_dataset = dataset["train"]
test_dataset = dataset["test"]

training arguments

In [7]:
from transformers import TrainingArguments, trainer
import torch

model.to(torch.device("cuda"))
training_args = TrainingArguments(
    num_train_epochs=3,
    output_dir="results",
)

trainer = trainer.Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=21, training_loss=11.267381940569196, metrics={'train_runtime': 6.7831, 'train_samples_per_second': 22.114, 'train_steps_per_second': 3.096, 'total_flos': 17413255987200.0, 'train_loss': 11.267381940569196, 'epoch': 3.0})